# How to use the bounding box

Skill level: Beginner

A bounding box specifies the spatial extent of an area of interest given its coordinates in meters:
- `x_min`: minimum x coordinate
- `y_min`: minimum y coordinate
- `x_max`: maximum x coordinate
- `y_max`: maximum y coordinate

View [API reference]

Run the following cells in order and do not skip any cells.<br />
If something seems off, just restart the runtime and run the cells again.

  [API reference]: https://geospaitial-lab.github.io/aviary/api_reference/core/bounding_box/#aviary.BoundingBox

## Install aviary

Install aviary in the current runtime using pip.

In [ ]:
! pip install -q geospaitial-lab-aviary

Import aviary and verify the installation.

In [ ]:
import aviary

print(aviary.__version__)

## Create a bounding box

You can pass the coordinates to the initializer of the bounding box.

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)

print(bounding_box)

We can visualize the bounding box in an interactive map with [Folium] for a better understanding.

Install folium in the current runtime using pip.

  [Folium]: https://python-visualization.github.io/folium

In [ ]:
! pip install -q folium

We define a function `visualize_bounding_box`, so that we can reuse it in the next steps.

In [ ]:
import folium
import geopandas as gpd


def visualize_bounding_box(
    bounding_box: aviary.BoundingBox,
    zoom_start: int = 16,
) -> folium.Map:
    # Convert the bounding box to a geodataframe
    gdf = bounding_box.to_gdf(epsg_code=25832)

    # Compute the centroid of the bounding box
    centroid = gpd.GeoDataFrame(
        geometry=[gdf.union_all().centroid],
        crs=gdf.crs,
    )

    # Convert the centroid to EPSG:4326 (Folium requires EPSG:4326)
    centroid_epsg_4326 = centroid.to_crs(epsg=4326)

    # Compute the location of the Folium map
    location_epsg_4326 = [
        centroid_epsg_4326.geometry.y.mean(),
        centroid_epsg_4326.geometry.x.mean(),
    ]

    # Convert the bounding box to EPSG:4326 (Folium requires EPSG:4326)
    gdf_epsg_4326 = gdf.to_crs(epsg=4326)

    # Create a Folium map
    folium_map = folium.Map(
        location=location_epsg_4326,
        zoom_start=zoom_start,
        tiles=None,
    )

    # Add OpenStreetMap tiles to the Folium map
    folium.TileLayer(
        tiles='OpenStreetMap',
        control=False,
    ).add_to(folium_map)

    # Define the style of the bounding box
    style_function = lambda feature: {
        'fillOpacity': .2,
        'color': 'black',
        'weight': 2,
    }

    # Add the bounding box to the Folium map
    folium.GeoJson(
        data=gdf_epsg_4326,
        style_function=style_function,
        control=False,
    ).add_to(folium_map)

    return folium_map

Now we can visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=bounding_box)

folium_map

You can access its coordinates using the [`x_min`][x_min], [`y_min`][y_min], [`x_max`][x_max], and [`y_max`][y_max] attributes.

  [x_min]: https://geospaitial-lab.github.io/aviary/api_reference/core/bounding_box/#aviary.BoundingBox.x_min
  [y_min]: https://geospaitial-lab.github.io/aviary/api_reference/core/bounding_box/#aviary.BoundingBox.y_min
  [x_max]: https://geospaitial-lab.github.io/aviary/api_reference/core/bounding_box/#aviary.BoundingBox.x_max
  [y_max]: https://geospaitial-lab.github.io/aviary/api_reference/core/bounding_box/#aviary.BoundingBox.y_max

In [ ]:
x_min = bounding_box.x_min
y_min = bounding_box.y_min
x_max = bounding_box.x_max
y_max = bounding_box.y_max

print(x_min)
print(y_min)
print(x_max)
print(y_max)

The bounding box is an iterable object, so it supports indexing and iterating.

You can access its coordinates using the index operator.

In [ ]:
x_min = bounding_box[0]
y_min = bounding_box[1]
x_max = bounding_box[2]
y_max = bounding_box[3]

print(x_min)
print(y_min)
print(x_max)
print(y_max)

You can also unpack its coordinates.

In [ ]:
x_min, y_min, x_max, y_max = bounding_box

print(x_min)
print(y_min)
print(x_max)
print(y_max)

You can iterate over its coordinates.

In [ ]:
for coordinate in bounding_box:
    print(coordinate)

The bounding box exposes its area via the [`area`][area] attribute.

  [area]: https://geospaitial-lab.github.io/aviary/api_reference/core/bounding_box/#aviary.BoundingBox.area

In [ ]:
area = bounding_box.area

print(area)

### Create a bounding box from a geodataframe

You can create a bounding box from a geodataframe using the [`from_gdf`][from_gdf] class method.

  [from_gdf]: https://geospaitial-lab.github.io/aviary/api_reference/core/bounding_box/#aviary.BoundingBox.from_gdf

In [ ]:
from shapely.geometry import box

gdf = gpd.GeoDataFrame(
    geometry=[box(363084, 5715326, 363340, 5715582)],
    crs='EPSG:25832',
)
bounding_box = aviary.BoundingBox.from_gdf(gdf=gdf)

print(bounding_box)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=bounding_box)

# Convert the geodataframe to EPSG:4326 (Folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the geodataframe (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#E7000B',
    'weight': 2,
}

# Add the geodataframe to the Folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

The geodataframe may contain multiple polygons, e.g., the districts of Gelsenkirchen.

In [ ]:
url = (
    'https://raw.githubusercontent.com/geospaitial-lab/aviary/main/docs'
    '/how_to_guides/api/data/districts.geojson'
)
gdf = gpd.read_file(url)
bounding_box = aviary.BoundingBox.from_gdf(gdf=gdf)

print(bounding_box)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(
    bounding_box=bounding_box,
    zoom_start=12,
)

# Convert the districts to EPSG:4326 (Folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the districts (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#E7000B',
    'weight': 2,
}

# Add the districts to the Folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

## Intersect bounding boxes

You can intersect two bounding boxes using the `&` operator.

In [ ]:
bounding_box_1 = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)
bounding_box_2 = aviary.BoundingBox(
    x_min=363212,
    y_min=5715454,
    x_max=363468,
    y_max=5715710,
)

print(bounding_box_1)
print(bounding_box_2)

In [ ]:
bounding_box = bounding_box_1 & bounding_box_2

print(bounding_box)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=bounding_box)

# Convert the first bounding box to a geodataframe
gdf_1 = bounding_box_1.to_gdf(epsg_code=25832)

# Convert the first bounding box to EPSG:4326 (Folium requires EPSG:4326)
gdf_1_epsg_4326 = gdf_1.to_crs(epsg=4326)

# Define the style of the first bounding box (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#E7000B',
    'weight': 2,
}

# Add the first bounding box to the Folium map
folium.GeoJson(gdf_1_epsg_4326, style_function=style_function).add_to(folium_map)

# Convert the second bounding box to a geodataframe
gdf_2 = bounding_box_2.to_gdf(epsg_code=25832)

# Convert the second bounding box to EPSG:4326 (Folium requires EPSG:4326)
gdf_2_epsg_4326 = gdf_2.to_crs(epsg=4326)

# Define the style of the second bounding box (blue)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#155DFC',
    'weight': 2,
}

# Add the second bounding box to the Folium map
folium.GeoJson(gdf_2_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

## Unite bounding boxes

You can unite two bounding boxes using the `|` operator.

In [ ]:
bounding_box_1 = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)
bounding_box_2 = aviary.BoundingBox(
    x_min=363212,
    y_min=5715454,
    x_max=363468,
    y_max=5715710,
)

print(bounding_box_1)
print(bounding_box_2)

In [ ]:
bounding_box = bounding_box_1 | bounding_box_2

print(bounding_box)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=bounding_box)

# Convert the first bounding box to a geodataframe
gdf_1 = bounding_box_1.to_gdf(epsg_code=25832)

# Convert the first bounding box to EPSG:4326 (Folium requires EPSG:4326)
gdf_1_epsg_4326 = gdf_1.to_crs(epsg=4326)

# Define the style of the first bounding box (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#E7000B',
    'weight': 2,
}

# Add the first bounding box to the Folium map
folium.GeoJson(gdf_1_epsg_4326, style_function=style_function).add_to(folium_map)

# Convert the second bounding box to a geodataframe
gdf_2 = bounding_box_2.to_gdf(epsg_code=25832)

# Convert the second bounding box to EPSG:4326 (Folium requires EPSG:4326)
gdf_2_epsg_4326 = gdf_2.to_crs(epsg=4326)

# Define the style of the second bounding box (blue)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#155DFC',
    'weight': 2,
}

# Add the second bounding box to the Folium map
folium.GeoJson(gdf_2_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

## Buffer the bounding box

You can expand the bounding box using the [`buffer`][buffer] method.

  [buffer]: https://geospaitial-lab.github.io/aviary/api_reference/core/bounding_box/#aviary.BoundingBox.buffer

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)

print(bounding_box)

In [ ]:
buffered_bounding_box = bounding_box.buffer(buffer_size=64)

print(buffered_bounding_box)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=buffered_bounding_box)

# Convert the original bounding box to a geodataframe
gdf = bounding_box.to_gdf(epsg_code=25832)

# Convert the original bounding box to EPSG:4326 (Folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the original bounding box (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#E7000B',
    'weight': 2,
}

# Add the original bounding box to the Folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

You can also shrink the bounding box using the [`buffer`][buffer] method.

  [buffer]: https://geospaitial-lab.github.io/aviary/api_reference/core/bounding_box/#aviary.BoundingBox.buffer

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)

print(bounding_box)

In [ ]:
buffered_bounding_box = bounding_box.buffer(buffer_size=-64)

print(buffered_bounding_box)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=buffered_bounding_box)

# Convert the original bounding box to a geodataframe
gdf = bounding_box.to_gdf(epsg_code=25832)

# Convert the original bounding box to EPSG:4326 (Folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the original bounding box (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#E7000B',
    'weight': 2,
}

# Add the original bounding box to the Folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

## Snap the bounding box

You can align the bounding box to a grid with the [`snap`][snap] method.

  [snap]: https://geospaitial-lab.github.io/aviary/api_reference/core/bounding_box/#aviary.BoundingBox.snap

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)

print(bounding_box)

In [ ]:
snapped_bounding_box = bounding_box.snap(value=128)

print(snapped_bounding_box)

Visualize the bounding box.

In [ ]:
folium_map = visualize_bounding_box(bounding_box=snapped_bounding_box)

# Convert the original bounding box to a geodataframe
gdf = bounding_box.to_gdf(epsg_code=25832)

# Convert the original bounding box to EPSG:4326 (Folium requires EPSG:4326)
gdf_epsg_4326 = gdf.to_crs(epsg=4326)

# Define the style of the original bounding box (red)
style_function = lambda feature: {
    'fillOpacity': 0.,
    'color': '#E7000B',
    'weight': 2,
}

# Add the original bounding box to the Folium map
folium.GeoJson(gdf_epsg_4326, style_function=style_function).add_to(folium_map)

folium_map

## Convert the bounding box to a geodataframe

You can convert the bounding box to a geodataframe using the [`to_gdf`][to_gdf] method.

  [to_gdf]: https://geospaitial-lab.github.io/aviary/api_reference/core/bounding_box/#aviary.BoundingBox.to_gdf

In [ ]:
bounding_box = aviary.BoundingBox(
    x_min=363084,
    y_min=5715326,
    x_max=363340,
    y_max=5715582,
)

print(bounding_box)

In [ ]:
gdf = bounding_box.to_gdf(epsg_code=25832)

print(gdf)